In [7]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from ipywidgets import interactive, interact, interactive_output

import ipywidgets as widgets

%matplotlib inline

AFPS = ['HABITAT', 'CUPRUM', 'CAPITAL', 'PLANVITAL', 'PROVIDA']


with open('processed_data/df_new_heroku.pkl', 'rb') as handle:
    data_out = pickle.load(handle)
    
#data_out = pd.read_pickle('processed_data/df_new_heroku.pkl')


def plot_m(**data):
    
    sel_data = ['Fecha']
    year = data['year']
    
    for serie in [1, 2, 3, 4, 5]:

        if data[f'c_{serie}']:
            afp = data[f'afp_{serie}']
            fondo = data[f'est_{serie}']
            lag = data[f'lag_{serie}']
            str_serie = 'V_{}_{}_lag_{}'.format(afp, fondo, lag)
            sel_data.append(str_serie)
            
        
    plt.figure(2)
    monto_inicial = 100

    df_sel = data_out[year][sel_data]

    df_sel.set_index('Fecha').plot()

    plt.title('Comparador')
    
    plt.show()

# Comparador de Fondos de Pensiones

Esta aplicación permite comparar la evolución de los fondos de pensiones chilenos, ya sea siguiendo una **estrategia "pasiva"** (vale decir, quedándose en alguno de los fondos A al E), o bien una **estrategia "activa"** (por ejemplo, seguir las recomendaciones de empresas como *Felices y Forrados*).

In [8]:
series = [1, 2, 3, 4, 5]
dict_wid = {}
hboxs = []
for i, serie in enumerate(series):
    if i == 0:
        sel = True
        fondo = 'A'
    elif i == 1:
        sel = True
        fondo = 'FF'
    else:
        sel = False
        fondo = 'A'
    check1 = widgets.Checkbox(
        value=sel,
        description='Serie {}'.format(serie),
        disabled=False,
        indent=False,
        layout=widgets.Layout(width='100px'),
    )
    drop1 = widgets.Dropdown(
        options=AFPS,
        value='HABITAT',
        description='AFP:',
        disabled=False,
    )
    drop2 = widgets.Dropdown(
        options=['A', 'B', 'C', 'D', 'E', 'FF', 'MAX'],
        value=fondo,
        description='Estrategia:',
        disabled=False,
    )
    drop3 = widgets.Dropdown(
        options=[0, 1, 2],
        value=0,
        description='Lag Solicitud:',
        disabled=False,
    )
    
    dict_wid[f'c_{serie}'] = check1
    dict_wid[f'afp_{serie}'] = drop1
    dict_wid[f'est_{serie}'] = drop2
    dict_wid[f'lag_{serie}'] = drop3

    items1 = [check1, drop1, drop2, drop3] 
    hboxs.append(widgets.HBox(items1))

year_wid = widgets.IntSlider(
    value=2014,
    min=2011,
    max=2020,
    step=1,
    description='Año de Inicio:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)
    
dict_wid['year'] = year_wid
hboxs.append(year_wid)

out = interactive_output(plot_m, dict_wid)

ui = widgets.VBox(hboxs)

accordion = widgets.Accordion(children=[ui])
accordion.set_title(0, 'Seleccione')

display(accordion, out)

Accordion(children=(VBox(children=(HBox(children=(Checkbox(value=True, description='Serie 1', indent=False, la…

Output()

## NOTAS

Para asegurar la comparabilidad en el tiempo, se asume que las personas hacen una única inversión en una fecha de inicio determinada y siguiendo una estrategia determinada. (*Un ejercicio más realista debería incorporar información de aportes (o retiros) mensuales y comisiones, pero quedará fuera del ámbito de este proyecto por el momento. En todo caso, las conclusiones principales no deberían ser afectadas por estos elementos.*).


Más antecedentes de la aplicación [AQUÍ](https://github.com/calvarad/ff).

Parámetros relevantes:
- **Estrategia**: Se refiere a una estrategia pasiva (dejar dinero en fondo A al E), o una estrategia activa (*FF* sería seguir las recomendaciones de Felices y Forrados). Estrategia "MAX" es una estrategia que al inicio de cada mes elije el fondo que tendrá más rentabilidad ese mes.
- **Lag Solicitud**: Se refiere al rezago que existe entre el momento en que se realiza la sugerencia de cambio de fondo y el momento en que la persona solicita el cambio de fondo. Por ejemplo, un Lag=2 significa que la persona se demora 2 días en solicitar formalmente el cambio de fondo a su AFP.

Para agregar más series al comparador, debe seleccionar la casilla **"Serie"** al costado izquierdo.